In [1]:
import pandas as pd  # to perform data manipulation and analysis
import numpy as np  # to cleanse data
import glob
from datetime import datetime  # to manipulate dates
import plotly.express as px  # to create interactive charts
import plotly.graph_objects as go  # to create interactive charts
from jupyter_dash import JupyterDash  # to build Dash apps from Jupyter environments
from dash import dcc  # to get components for interactive user interfaces
from dash import html  # to compose the dash layout using Python structures\

In [2]:
# List of retailers in each category
restaurants = [
    'lazeez',
    'shin wa',
    'tim hortons',
    'timhortons',
    'ubereats',
    'uber eats',
    'skipthedishes',
    'doordash',
    'door dash',
    'jugo juice',
    'swiss chalet',
    'starbucks',
    'mcdonald',
    'fantastic wok',
    'marble slab',
    'the bauer kitchen',
    'booster juice',
    'housecook',
    'meltwich',
    'dairy queen',
    'dairyqueen',
    'kabob',
    'shawerma',
    'bk',
    'burger king',
    'burgerking',
    'krispy',
    'eggspectation',
    'ben & jerry',
    'popeyes',
    'creelman hall',
    'harvey\'s'
]
rslt_restaurants = '|'.join(restaurants)

grocery = [
    'wal*mart',
    'wal-mart',
    'walmart',
    'kishki',
    'wmt suprctr',
    'dollarama',
    'dollar shave',
    'canadian tire',
    'canadiantire',
    'cdn tire',
    'london drug', 
    'sherwood co-op'
]
rslt_grocery = '|'.join(grocery)

clothing = [
    'gymshark',
    'oldnavy',
    'old navy',
    'hudson\'s bay',
    'marshalls',
    'h\&m',
    'h \& m',
    'bed bath',
    'sportchek',
    'on_canada',
    'atmosphere',
    'aeo'
]
rslt_clothing = '|'.join(clothing)

shopping = [
    'etsy'
]
rslt_shopping = '|'.join(shopping)

In [3]:
path =  "./csvFiles"
cibc_files = glob.glob(path + "/cibc*.csv")
tang_files = glob.glob(path + "/tang*.csv")

In [4]:
df = pd.concat((pd.read_csv(f) for f in cibc_files))
df.sort_values(by="Date")
df.rename(columns={'Transactions': 'Transaction'}, inplace=True)
pd.set_option('display.max_rows', 20)
df.head()
# print(df.loc[[1]])

,Date,Transaction,Debit,Credit
0,2021-12-22,"CINEPLEX 8030 WEB QPS 416-323-6600, ON",27.04,NaN
1,2021-12-20,"PAC-WESTJETCONNECT LAKE FOREST, CA",14.68,NaN
2,2021-12-20,"NETFLIX.COM 844-5052993, BC",16.64,NaN
3,2021-12-20,"WESTJET 8382607508085 CALGARY, AB",33.90,NaN
4,2021-12-17,PAYMENT THANK YOU/PAIEMEN T MERCI,NaN,171.07


In [5]:
df2 = pd.concat((pd.read_csv(f, encoding='latin1') for f in tang_files))
df2.sort_values(by="Date")
df2.drop('Type', axis=1,inplace=True)
df2.head()


,Date,Transaction,Rewards,Amount
0,2021-12-20,PAYMENT - THANK YOU,NaN,250.00
1,2021-12-20,DR KELLY RUSTEIKA DENT REGINA SK,Rewards earned: 1.00 ~ Category: Other,-199.00
2,2021-12-20,UBER CANADA/UBERTRIP TORONTO ON,Rewards earned: 0.05 ~ Category: Parking,-9.71
3,2021-12-17,JUGO JUICE CALGARY AB,Rewards earned: 0.06 ~ Category: Restaurant,-3.06
4,2021-12-17,LAZEEZ SHAWARMA WATERLOO ON,Rewards earned: 0.34 ~ Category: Restaurant,-17.20


In [6]:
conditions = [
    (df['Credit'].notnull()),
    (df['Debit'].notnull()),
]
values = [df['Credit'],df['Debit'] *-1]
df['Amount'] = np.select(conditions, values)
df.drop('Debit', axis=1,inplace=True)
df.drop('Credit', axis=1, inplace=True)
df.head()
# print(df.loc[[1]])

,Date,Transaction,Amount
0,2021-12-22,"CINEPLEX 8030 WEB QPS 416-323-6600, ON",-27.04
1,2021-12-20,"PAC-WESTJETCONNECT LAKE FOREST, CA",-14.68
2,2021-12-20,"NETFLIX.COM 844-5052993, BC",-16.64
3,2021-12-20,"WESTJET 8382607508085 CALGARY, AB",-33.90
4,2021-12-17,PAYMENT THANK YOU/PAIEMEN T MERCI,171.07


In [7]:
def category(x):
    return x.partition("Category: ")[2] if x != "" else ""

df2['Rewards'].fillna("", inplace=True)
df2['Category'] = df2['Rewards'].apply(category)
df2.drop("Rewards", axis=1, inplace=True)

In [8]:
df['Category'] = 'nan'
df3 = pd.concat([df, df2], axis=0)
df3.sort_values(by="Date", inplace=True)
df3 = df3.applymap(lambda s: s.lower() if type(s) == str else s)
df3['Transaction'] = df3['Transaction'].replace({'\*':''}, regex=True)
# print(df)

In [9]:
df3['Category'] = np.where(df3['Transaction'].str.contains('amazon|amz'),'Online Shopping', df3['Category'])
df3['Category'] = np.where(df3['Transaction'].str.contains('presto|transit|go transit|rmow grt'), 'Transit', df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains('uber|bixi'), 'Transport', df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains('spotify|netflix|prime|bear|icloud|curiositystream'),'Subscriptions', df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains('westjet|air canada|aircanada|aeroplan|air can'),'Flights', df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains(rslt_restaurants),'Eating Out', df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains(rslt_grocery), 'Grocery', df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains(rslt_shopping),'Shopping' ,df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains('coinamatic'),'laundry' ,df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains('pharma'),'Pharmacy' ,df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains(rslt_clothing),'Clothing' ,df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains('credit card rewards'),'Rewards Income' ,df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains('airbnb|marriot|hilton'),'Travel Lodging' ,df3['Category'] )
df3['Category'] = np.where(df3['Category'].str.contains('restaurant'),'Eating Out' ,df3['Category'] )
df3['Category'] = np.where(df3['Category'].str.contains('groceries'),'Grocery' ,df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains('passion salon'),'Haircut' ,df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains('cineplex'),'Entertainment' ,df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains('electronic funds transfer pay'),'Income' ,df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains('best buy|fitbit|apple'),'Tech' ,df3['Category'] )
df3['Category'] = np.where(df3['Transaction'].str.contains('islam'),'Charity' ,df3['Category'] )
df3['Category'] = np.where(df3['Category'].isna(),'Other' ,df3['Category'] )

In [10]:
# E-Transfer INTO my account
trans_in_cond = (df3['Transaction'].str.contains('e-transfer')) & (df3['Amount'] > 0)
df3.loc[trans_in_cond, 'Category'] = 'Transfer In'

# E-Transfer OUT of my account
trans_out_cond = (df3['Transaction'].str.contains('e-transfer')) & (df3['Amount'] < 0)
df3.loc[trans_out_cond, 'Category'] = 'Transfer Out'

# Electronic Funds Transfer In
trans_in_cond2 = (df3['Transaction'].str.contains('electronic funds transfer')) & (df3['Amount'] > 0) & ~(df3['Transaction'].str.contains('pay'))
df3.loc[trans_in_cond2, 'Category'] = 'Transfer In'

# Electronic Funds Transfer Out
trans_out_cond2 = (df3['Transaction'].str.contains('electronic funds transfer')) & (df3['Amount'] < 0)
df3.loc[trans_out_cond2, 'Category'] = 'Transfer Out'